In [7]:
# wir benötigen wieder ein paar python imports...
import sys
from os import environ
import os
import uuid
sys.path.append(os.path.abspath(".."))

from sparql_queries import *
from connector_client import ConnectorClient
from connector.messages.datamodel_utils import SubscriptionRegisterRequest, SubscriptionUnregisterRequest # generic messages within the connector_base repo

client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])

In [8]:
# graphdb queries, um den richtigen connector zu identifizieren
await client.load_connector_config()
await client.return_connectors()

dict_keys(['opcua_testserver'])

In [9]:
# wechsel an den ausgewählten, richtigen connector
await client.switch_connector('opcua_testserver')

In [10]:
# subscription_id aus dem Knowledge Graph "ziehen"
output = await client.query_graphdb(build_subscription_query(device_identifier="CX2030"))
print(output)

                           subscription                                device  \
0  644c6a2f-fed8-4b2a-9d3f-d543a4fcf21e  d91ac2c8-0a1e-4fe0-b1aa-b5e6f2d3579c   

                              datapoint                       start   state  
0  33856881-41f7-465d-b41a-d98a23541572  2025-04-27T20:52:52.142910  Active  


In [11]:
subscription_id = output.loc[0, "subscription"]
# definieren einer Unsubscribe-Nachricht für einen OPC-UA-Server
example_unsubscribe_payload = SubscriptionUnregisterRequest(
    subscription_identifier = subscription_id,
    device_origin="client"
)

In [12]:
await client.unsubscribe_data_source(base_payload = example_unsubscribe_payload)

message type: <class 'connector.messages.datamodel_utils.SubscriptionClosedResponse'>
Matching response received:
{
    "type": "SubscriptionClosedResponse",
    "device_origin": "module_opcua",
    "subscription_identifier": "644c6a2f-fed8-4b2a-9d3f-d543a4fcf21e",
    "status": "success"
}


MsgModel(root=ResponseMessage(version='1.0.0', payload=CommandResponse(type='CommandResponse', base_payload=SubscriptionClosedResponse(type='SubscriptionClosedResponse', device_origin='module_opcua', subscription_identifier=UUID('644c6a2f-fed8-4b2a-9d3f-d543a4fcf21e'), status='success')), type='ResponseMessage', message_id=UUID('d4f7e870-e6ff-4f06-aa42-670709087f0b'), correlation_id=UUID('11281ba8-06d5-407f-976c-0a87559f036d'), timestamp=datetime.datetime(2025, 4, 27, 20, 53, 2, 233220), status_code='Valid', service_id='opcua_testserver'))